In [1]:
from typing import List, Optional, Union, Tuple
from pydantic import BaseModel, Field
from abstractions.goap.actions import Prerequisites, Goal
from abstractions.goap.entity import Statement, Attribute
from abstractions.goap.nodes import GameEntity, Node, Position
from abstractions.goap.language_state import GoalState
from abstractions.goap.gridmap import GridMap
from abstractions.goap.interactions import Character, Door, Key, Treasure, Floor, Wall, InanimateEntity, IsPickupable, TestItem, Open, Close, Unlock, Lock, Pickup, Drop, Move
from abstractions.goap.game.main import generate_dungeon
from abstractions.goap.shapes import Rectangle, Shadow, Radius, RayCast, BlockedRaycast



def source_target_position_comparison(source: tuple[int,int], target: tuple[int,int]) -> bool:
    """Check if the source entity's position is the same as the target entity's position."""
    if source and target:
        return source == target
    return False

def treasure_in_neighborhood(source: GameEntity, target: Optional[GameEntity] = None) -> bool:
    """Check if the treasure is in the character's neighborhood."""
    if source.node and target and target.node:
        return source.node in target.node.neighbors()
    return False

def key_in_inventory(source: GameEntity, target: Optional[GameEntity] = None) -> bool:
    """Check if the key is in the character's inventory."""
    if target:
        return target in source.inventory
    return False

def is_treasure(source: GameEntity, target: Optional[GameEntity] = None) -> bool:
    """Check if the target entity is a Treasure."""

    
    return isinstance(target, Treasure)

def is_golden_key(source: GameEntity, target: Optional[GameEntity] = None) -> bool:
    """Check if the entity is a Golden Key."""
    return isinstance(target, Key) and target.key_name.value == "Golden Key"

def is_door(entity: GameEntity, target: Optional[GameEntity] = None) -> bool:
    """Check if the entity is a Door."""
    return isinstance(entity, Door)

# Example usage
grid_map = GridMap(width=10, height=10)
grid_map.register_actions([Move, Pickup, Drop, Open, Close, Unlock, Lock])
room_width, room_height = 6, 6
character, door, key, treasure = generate_dungeon(grid_map, room_width, room_height)
grid_map.generate_entity_type_map()

        # Goal 1: Check if the character's position is the same as the treasure's position
reach_treasure_goal = Goal(
    name="Reach the treasure",
    source_entity_id=character.id,
    target_entity_id=treasure.id,
    prerequisites=Prerequisites(
        source_statements=[Statement(conditions={"can_act": True})],
        target_statements=[Statement(callables=[is_treasure])],
        source_target_statements=[
            Statement(comparisons={
                "source_position": ("position", "position", source_target_position_comparison)
            })
        ]
    )
)

# Goal 2: Check if the treasure is in the character's neighborhood
treasure_in_neighborhood_goal = Goal(
    name="Treasure in neighborhood",
    source_entity_id=character.id,
    target_entity_id=treasure.id,
    prerequisites=Prerequisites(
        source_statements=[Statement(conditions={"can_act": True})],
        target_statements=[Statement(callables=[is_treasure])],
        source_target_statements=[
            Statement(callables=[treasure_in_neighborhood])
        ]
    )
)

# Goal 3: Check if the key is in the character's inventory
key_in_inventory_goal = Goal(
    name="Key in inventory",
    source_entity_id=character.id,
    target_entity_id=key.id,
    prerequisites=Prerequisites(
        source_statements=[Statement(conditions={"can_act": True})],
        target_statements=[Statement(callables=[is_golden_key])],
        source_target_statements=[
            Statement(callables=[key_in_inventory])
        ]
    )
)

# Goal 4: Check if the door is unlocked
door_unlocked_goal = Goal(
    name="Door unlocked",
    source_entity_id=door.id,
    prerequisites=Prerequisites(
        source_statements=[
            Statement(callables=[is_door]),
            Statement(conditions={"is_locked": False})
        ],
        target_statements=[],
        source_target_statements=[]
    )
)

goals = [reach_treasure_goal, treasure_in_neighborhood_goal, key_in_inventory_goal, door_unlocked_goal]
# Add the goals to the GoalState
goal_state = GoalState(character_id=character.id, goals=goals)


# Generate goal information based on different shapes
rectangle = grid_map.get_rectangle()
goal_info_rectangle = goal_state.generate(rectangle)
print("Goal Information (Rectangle):")
print(goal_info_rectangle)

shadow = grid_map.get_shadow(source=character.node, max_radius=5)
goal_info_shadow = goal_state.generate(shadow)
print("\nGoal Information (Shadow):")
print(goal_info_shadow)

radius = grid_map.get_radius(source=character.node, max_radius=5)
goal_info_radius = goal_state.generate(radius)
print("\nGoal Information (Radius):")
print(goal_info_radius)

pygame-ce 2.4.1 (SDL 2.28.5, Python 3.10.6)
Updating block attributes... for door
Goal Information (Rectangle):
# Agent Goals
## Goal: Reach the treasure
### Spatial Information:
- Distance from Character to Treasure: 5
- Path from Character to Treasure: No path found
- Blocked Ray:  (Blocked by Door at (5, 2), Attributes: BlocksMovement: True, BlocksLight: True, Material: , Open: False, is_locked: True, required_key: Golden Key)
### Source statements:
- Conditions:
  - can_act: Satisfied (Desired: True, Actual: True)
### Target statements:
- Callables:
  - Check if the target entity is a Treasure.: Satisfied
### Source target statements:
- Comparisons:
  - source_position: Not Satisfied (Source: (5, 1), Target: (5, 6))

## Goal: Treasure in neighborhood
### Spatial Information:
- Distance from Character to Treasure: 5
- Path from Character to Treasure: No path found
- Blocked Ray:  (Blocked by Door at (5, 2), Attributes: BlocksMovement: True, BlocksLight: True, Material: , Open: False